# Projeto: Assistente de Viagem

Neste notebook vamos combinar todos os conceitos vistos ate agora para construir um agente funcional: um **assistente de viagem** que ajuda o usuario a planejar destinos.

O agente vai usar:

- **System prompt** para definir sua persona e comportamento
- **Tool** para buscar informacoes atualizadas na web
- **Memoria** para manter o contexto entre as mensagens

O objetivo e mostrar como esses blocos fundamentais se combinam para criar um agente util e funcional.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Definindo a tool de busca

O assistente precisa buscar informacoes atualizadas sobre destinos, precos de passagens, clima e atracoes. Para isso, vamos reutilizar a tool de busca na web com Tavily.

In [2]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient

tavily_client = TavilyClient()

@tool
def buscar_na_web(query: str) -> Dict[str, Any]:
    """Busca informacoes atualizadas na internet."""
    return tavily_client.search(query)

## Definindo o system prompt

Um bom system prompt define claramente o papel do agente, suas capacidades e o formato esperado das respostas. Quanto mais especifico o prompt, mais consistente e util sera o comportamento do agente.

In [3]:
system_prompt = """
Voce e um assistente de viagem experiente e atencioso.

Seu papel e ajudar o usuario a planejar viagens, oferecendo sugestoes de destinos, roteiros, dicas praticas e informacoes uteis.

Regras:
- Sempre use a ferramenta de busca para obter informacoes atualizadas sobre precos, clima e eventos
- Seja objetivo e organize as informacoes de forma clara
- Quando sugerir roteiros, inclua estimativas de tempo e custo quando possivel
- Lembre-se das preferencias que o usuario mencionou durante a conversa
"""

## Criando o agente

Agora combinamos tudo: modelo, tools, system prompt e checkpointer para memoria.

In [4]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agente = create_agent(
    model="gpt-4.1-nano",
    tools=[buscar_na_web],
    system_prompt=system_prompt,
    checkpointer=InMemorySaver()
)

In [5]:
config = {"configurable": {"thread_id": "viagem-1"}}

## Interagindo com o agente

Vamos simular uma conversa real com o assistente. Cada chamada a seguir faz parte da mesma conversa (mesmo `thread_id`), entao o agente vai acumulando contexto.

In [6]:
from langchain.messages import HumanMessage

resposta = agente.invoke(
    {"messages": [HumanMessage(content="Estou pensando em viajar para o Japao em outubro. O que voce sugere?")]},
    config
)

print(resposta["messages"][-1].content)

Em outubro, o Japão oferece uma experiência espetacular devido às cores do outono, conhecidas como "koyo". Algumas sugestões de destinos e atividades são:

1. **Kyoto** - famosa pelos parques, templos e jardins que ficam ainda mais belos com as folhas vermelhas e douradas. Vale fazer o passeio de "Momijigari", a apreciação das folhas de outono.

2. **Lago Tazawa em Akita, Lagoa Tsutanuma em Aomori e Hachimantai** - locais populares para apreciar as cores do outono e fazer caminhadas.

3. **Senjojiki Cirque, em Nagano** - ideal para trilhas e contato com a natureza.

4. **Festivais e tradições** - outubro é época de celebrações culturais, de colheitas como castanhas, batata-doce e abóbora, além de cerimônias de chá.

5. **Roteiro sugerido**: explorar Koyasan por 3 noites, usar Kyoto como base para explorar localidades próximas como Uji e a trilha Kurama–Kibune.

Quer que eu ajude a montar um roteiro detalhado, incluindo duração, custo estimado, ou dicas de hospedagem?


In [7]:
resposta = agente.invoke(
    {"messages": [HumanMessage(content="Meu orcamento e de 15 mil reais, incluindo passagem. Da pra fazer?")]},
    config
)

print(resposta["messages"][-1].content)

Vamos fazer uma estimativa para verificar se é possível planejar sua viagem ao Japão com um orçamento de 15 mil reais, incluindo passagem aérea, hospedagem, alimentação e passeios.  

Primeiro, preciso saber algumas informações adicionais:  
- Para qual cidade no Japão você planeja viajar (Tokyo, Kyoto, Osaka, etc.)?  
- Quantos dias você pretende ficar?  
- Você tem preferência por algum tipo de hospedagem (hotel, hostel, ryokan)?  
- Há interesses específicos que possam influenciar custos, como visitas a templos, parques ou eventos?  

Com esses dados, poderei fazer uma estimativa mais precisa.


In [8]:
resposta = agente.invoke(
    {"messages": [HumanMessage(content="Monta um roteiro de 7 dias pra mim focando em Tokyo e Kyoto.")]},
    config
)

print(resposta["messages"][-1].content)

Com base nas buscas, aqui estão as principais informações para seu roteiro de 7 dias focado em Tóquio e Kyoto, dentro do seu orçamento de 15 mil reais:

### Passagens aéreas
- Voos de ida e volta para Tóquio em outubro podem variar, com opções econômicas a partir de aproximadamente R$ 1.900 a R$ 2.500, dependendo da antecedência e da companhia aérea. Recomendo reservar com antecedência para melhores preços.

### Hospedagem
- Hotéis econômicos em Tóquio e Kyoto custam em média entre R$ 150 a R$ 300 por noite. Uma opção boa e acessível é procurar hostels ou guesthouses, que custam cerca de R$ 100 a R$ 200 por noite.

### Roteiro de 7 dias sugerido
**Dia 1:** Chegada a Tóquio   
- Hospedagem: Hostel ou hotel econômico  
- Atividades: Chegada, check-in, passeio no bairro de Shinjuku ou Shibuya para conhecer a cidade à noite  

**Dia 2:** Tóquio  
- Visitar o Senso-ji em Asakusa, Ponte de Rainbow Bridge, e o bairro de Ginza  
- Opcional: Visitar o parque Ueno e museus próximos  

**Dia 3:**

In [9]:
resposta = agente.invoke(
    {"messages": [HumanMessage(content="Qual o meu orcamento mesmo? E pra quando eu quero ir?")]},
    config
)

print(resposta["messages"][-1].content)

Seu orçamento total para a viagem ao Japão é de 15 mil reais, incluindo passagem aérea, hospedagem, alimentação, transporte e passeios.  

Você mencionou que deseja viajar em outubro.  

Se desejar, posso ajudar a ajustar o roteiro, otimizar os custos ou planejar a viagem para aproveitar ao máximo dentro desse orçamento.


Essa ultima pergunta confirma que a memoria esta funcionando. O agente lembra do orcamento e do periodo mencionados nas mensagens anteriores, sem que tenhamos passado essas informacoes novamente.

Com apenas algumas linhas de codigo, combinamos modelo, tools, prompting e memoria para criar um assistente funcional. Nas proximas aulas, vamos expandir esses conceitos com fluxos mais complexos usando LangGraph.